In [1]:
import numpy as np
import scipy as sp
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve

import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt

import seaborn as sns

Ler o dataset

In [2]:
data_set = pd.read_csv("data/trn.csv", sep=',', header=0, low_memory=False)

Pre-processamento básico

In [3]:
# encontra as variavies categoricas
for col in data_set.columns:
    if ((data_set[col].quantile() == 1.0) | (data_set[col].quantile() == 0.0)):
        data_set[col] = data_set[col].astype('category')

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X = data_set.loc[:, data_set.columns !='y']
y = data_set.iloc[:, data_set.columns == 'y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/4, random_state=42, stratify=y)

In [6]:
from sklearn.ensemble import RandomForestClassifier

In [7]:
from sklearn.model_selection import RandomizedSearchCV

Todas as combinações de parametros interessates

In [8]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

{'n_estimators': [100, 311, 522, 733, 944, 1155, 1366, 1577, 1788, 2000], 'min_samples_split': [2, 5, 10], 'bootstrap': [True, False], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_leaf': [1, 2, 4], 'max_features': ['auto', 'sqrt']}


Validação cruzada para encontrar a melhor combinação de hyperparametros
Usando 3-fold

In [ ]:
rf = RandomForestClassifier(random_state = 402)

# procura atraves de 100 combinações diferentes
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,\
                                n_iter = 10, scoring='neg_mean_absolute_error',\
                                cv = 3, verbose=2, random_state=402, n_jobs=-1,\
                                return_train_score=True)

# Fit a busca aleatoria
rf_random.fit(X_train.values, y_train.values.ravel());

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] n_estimators=2000, min_samples_split=10, bootstrap=True, max_depth=30, min_samples_leaf=1, max_features=auto 


Process ForkPoolWorker-2:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/wba/.local/lib/python3.5/site-packages/sklearn/externals/joblib/pool.py", line 362, in get
    return recv()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 251, in recv
    return ForkingPickler.loads(buf.getbuffer())
MemoryError


[CV] n_estimators=2000, min_samples_split=10, bootstrap=True, max_depth=30, min_samples_leaf=1, max_features=auto 
